In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp 'kaggle.json' ~/.kaggle
!chmod 600  ~/.kaggle/'kaggle.json'
!kaggle datasets download -d 'misrakahmed/vegetable-image-dataset'

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset
License(s): CC-BY-SA-4.0
100% 534M/534M [00:04<00:00, 81.7MB/s]
100% 534M/534M [00:04<00:00, 112MB/s] 


##importing library

In [ ]:
!pip freeze > requirements.txt
import zipfile
import os
import shutil
import  tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataset_zip=zipfile.ZipFile('vegetable-image-dataset.zip')
dataset_zip.extractall()
dataset_zip.close()

In [ ]:

training_dir = os.path.join('Vegetable Images', 'train')
validation_dir = os.path.join('Vegetable Images', 'validation')

In [ ]:
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
else:
    print(f"Directory '{validation_dir}' already exists.")

Directory 'Vegetable Images/validation' already exists.


In [ ]:
validation_dataset_portion = 0.01 # 1% -> 1/100 -> 0.001

In [ ]:
for class_dir_name in os.listdir(training_dir):
    print(class_dir_name)
    class_dir_path = os.path.join(training_dir, class_dir_name)


    #cari banyak data training
    train_dataset_num = len(os.listdir(class_dir_path))

    #cari banyak data validation
    val_dataset_num = len(os.listdir(os.path.join(validation_dir, class_dir_name)))

    #buat  direktori kelas direktori validation
    class_val_dir = os.path.join(validation_dir, class_dir_name)
    if not os.path.exists(class_val_dir):
        os.mkdir(class_val_dir)

    counter = 0

    for filename in os.listdir(class_dir_path):
        filepath = os.path.join(class_dir_path, filename)

        if counter < val_dataset_num:
          destination_path = os.path.join(class_val_dir, filename)
          shutil.move(filepath, destination_path)

        counter += 1


Bottle_Gourd
Potato
Tomato
Bitter_Gourd
Broccoli
Capsicum
Cauliflower
Brinjal
Papaya
Cabbage
Bean
Cucumber
Pumpkin
Radish
Carrot


##image generator

In [ ]:
training = ImageDataGenerator(
    rescale = 1/255,
    horizontal_flip = True

)
validation = ImageDataGenerator(
    rescale = 1/255
)

In [ ]:
training_generator = training.flow_from_directory(
    training_dir,
    target_size = (224,224),
    class_mode = 'categorical',
    batch_size = 20,
    shuffle = True
)

validation_generator = validation.flow_from_directory(
    validation_dir,
    target_size = (224,224),
    class_mode = 'categorical',
    batch_size = 20,
    shuffle = True
)

Found 12000 images belonging to 15 classes.
Found 6000 images belonging to 15 classes.


##model

In [ ]:
model = tf.keras.models.Sequential ([
    tf.keras.layers.Conv2D(8,(3,3),activation='relu',input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(15,activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
model.fit(
    training_generator,
    epochs = 5,
    validation_data = validation_generator
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


600/600 ━━━━━━━━━━━━━━━━━━━━ 867s 1s/step - accuracy: 0.4134 - loss: 3.0023 - val_accuracy: 0.7495 - val_loss: 0.8069
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 953s 1s/step - accuracy: 0.8037 - loss: 0.6412 - val_accuracy: 0.7770 - val_loss: 0.7475
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 881s 1s/step - accuracy: 0.8982 - loss: 0.3403 - val_accuracy: 0.7827 - val_loss: 0.7805
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 892s 1s/step - accuracy: 0.9351 - loss: 0.2360 - val_accuracy: 0.8125 - val_loss: 0.6264
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 899s 1s/step - accuracy: 0.9485 - loss: 0.1740 - val_accuracy: 0.8420 - val_loss: 0.5518


save keras

In [ ]:
import tensorflow as tf

model.save('saved_model.keras')

##download tflite

In [ ]:

import tensorflow as tf

# Convert the Keras model to a TensorFlow Lite model
# Change to load from the HDF5 file directly using `from_keras_model`
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

from google.colab import files
files.download('model.tflite')

Saved artifact at '/tmp/tmps122qomx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 15), dtype=tf.float32, name=None)
Captures:
  138214182976000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138214131699376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138214131697264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138214181615072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138214168357952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138214131711520: TensorSpec(shape=(), dtype=tf.resource, name=None)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf
model.save('model.h5')
my_model = tf.keras.models.load_model('model.h5')

##save h5


In [ ]:
from tensorflow.python.saved_model import save

save_dir = os.path.join('model/saved_model.h5')
model.save(save_dir)

##Convert js

In [ ]:
!pip3 install tensorflowjs

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully unin

In [ ]:
!cd model/saved_model.h5
!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model 'model/saved_model.h5' '/content/tfjs_model'

/bin/bash: line 1: cd: model/saved_model.h5: Not a directory
2024-12-12 06:28:35.979156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733984916.009082   23525 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733984916.018128   23525 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file


In [ ]:
!zip -r /content/model.zip /content

Output streaming akan dipotong hingga 5000 baris terakhir.
  adding: content/Vegetable Images/validation/Tomato/1308.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1318.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1359.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1362.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1262.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1341.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1276.jpg (deflated 53%)
  adding: content/Vegetable Images/validation/Tomato/1310.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1282.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1286.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1307.jpg (deflated 0%)
  adding: content/Vegetable Images/validation/Tomato/1355.jpg (deflated 51%)
  adding: content/Vegetable

In [ ]:
!zip -r /content/tfjs_model.zip /content/tfjs_model

  adding: content/tfjs_model/ (stored 0%)
  adding: content/tfjs_model/group1-shard46of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard22of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard45of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard27of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard47of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard38of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard48of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard10of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard9of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard5of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard28of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard19of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard42of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-shard2of49.bin (deflated 7%)
  adding: content/tfjs_model/group1-s

##coba model

In [ ]:
my_model.get_config


<bound method Sequential.get_config of <Sequential name=sequential, built=True>>

In [ ]:
image_filename = 'download.jpg'

In [ ]:
image = tf.keras.utils.load_img(image_filename, target_size=(224, 224))

In [ ]:
image_array = tf.keras.utils.img_to_array(image)

In [ ]:
image_array.shape

(224, 224, 3)

In [ ]:
image_array = tf.expand_dims(image_array, 0)

In [ ]:
import numpy as np

In [ ]:
batch_image_array = image_array

In [ ]:
batch_image_array.shape

TensorShape([1, 224, 224, 3])

In [ ]:
my_model.predict(batch_image_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [ ]:
result = my_model.predict(batch_image_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


In [ ]:
result

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [ ]:
index = 0
for number in result[0]:
  if number > 0:
    print(index)
  index += 1

6


In [ ]:
training_generator.class_indices

{'Bean': 0,
 'Bitter_Gourd': 1,
 'Bottle_Gourd': 2,
 'Brinjal': 3,
 'Broccoli': 4,
 'Cabbage': 5,
 'Capsicum': 6,
 'Carrot': 7,
 'Cauliflower': 8,
 'Cucumber': 9,
 'Papaya': 10,
 'Potato': 11,
 'Pumpkin': 12,
 'Radish': 13,
 'Tomato': 14}